In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def se_block(x, r=16):
    c = int(x.shape[-1])
    s = layers.GlobalAveragePooling2D()(x)               
    print('GlobalAveragePooling2D.shape:', s.shape)
    s = layers.Dense(units= c//r, activation='relu')(s) 
    print('Dense1.shape:', s.shape)
    s = layers.Dense(units=c, activation='sigmoid')(s)           
    print('Dense2.shape:', s.shape)
    s = layers.Reshape((1, 1, c))(s)                       
    print('Reshape.shape:', s.shape)
    return layers.Multiply()([x, s])
                  

def bottleneck_se(x, stride=2):
    shortcut = x
    # projection shortcut
    if stride != 1:
        shortcut = layers.Conv2D(filters=256, kernel_size=1, strides=stride, use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    print('shortcut.shape:', shortcut.shape)
    
    x = layers.Conv2D(filters=64, kernel_size=1, use_bias=False)(x)
    print('x1.shape:', x.shape)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=stride, padding='same', use_bias=False)(x)
    print('x2.shape:', x.shape)    
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters=256, kernel_size=1, use_bias=False)(x)
    print('x3.shape:', x.shape)    
    x = layers.BatchNormalization()(x)

    x = se_block(x)  
    print('x4.shape:', x.shape)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

inputs = layers.Input((112, 112, 64))    
outputs = bottleneck_se(inputs, stride=2)
model = models.Model(inputs, outputs)
model.summary()

shortcut.shape: (None, 56, 56, 256)
x1.shape: (None, 112, 112, 64)
x2.shape: (None, 56, 56, 64)
x3.shape: (None, 56, 56, 256)
GlobalAveragePooling2D.shape: (None, 256)
Dense1.shape: (None, 16)
Dense2.shape: (None, 256)
Reshape.shape: (None, 1, 1, 256)
x4.shape: (None, 56, 56, 256)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 112, 112,  │          0 │ -                 │
│ (InputLayer)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 112, 112,  │      4,096 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_6 (ReLU)      │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 56, 56,    │     36,864 │ re_lu_6[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_7 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 56, 56,    │     16,384 │ re_lu_7[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │      4,112 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │      4,352 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 1, 1, 256) │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 56, 56,    │     16,384 │ input_layer_2[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_2          │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Multiply)          │ 256)              │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 56, 56,    │          0 │ multiply_2[0][0], │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_8 (ReLU)      │ (None, 56, 56,    │          0 │ add_2[0][0]     

 Total params: 84,752 (331.06 KB)

 Trainable params: 83,472 (326.06 KB)

 Non-trainable params: 1,280 (5.00 KB)